In [ ]:
# import modules

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors, cm
from pathlib import Path
import os
import gstools as gs
from collections import OrderedDict

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature
from cartopy.io import shapereader
from cartopy.io.img_tiles import Stamen
from cartopy.io.img_tiles import GoogleTiles
from owslib.wmts import WebMapTileService
from matplotlib.transforms import offset_copy
from scipy.interpolate import interp1d

import networkx as nx
import json

parent_dir = str(Path(os.path.dirname(os.path.realpath("__file__"))).parents[0])


In [ ]:
# Load data

# Load all transition matrices
transitionmatrix = np.load('output/Tm_tbeach5.npy')
TM1 = np.load('output/Tm_tbeach1.npy')
TM2 = np.load('output/Tm_tbeach2.npy')
TM10 = np.load('output/Tm_tbeach10.npy')
TM26 = np.load('output/Tm_tbeach26.npy')
TM35 = np.load('output/Tm_tbeach35.npy')
TM100 = np.load('output/Tm_tbeach100.npy')
TM_all = np.load('output/Tm_all_particles.npy')
TM_half = np.load('output/Tm_half_particles.npy')

# load grid specifics
gridsdataframe = pd.read_csv(parent_dir + '/particle_simulation/input/GridsDataFrame.csv')
gridnumbermask = np.load(parent_dir + '/particle_simulation/input/gridnumbermask.npy')
gridlon = np.load(parent_dir + '/particle_simulation/input/x_corners_grid.npy')
gridlat = np.load(parent_dir + '/particle_simulation/input/y_corners_grid.npy')
lat_release = np.load(parent_dir + '/particle_simulation/input/ReleaseLat.npy')
lon_release = np.load(parent_dir + '/particle_simulation/input/ReleaseLon.npy')
endloc = np.load('output/endloc.npy')

# number of coastal grids
c_total = len(gridsdataframe) 

# number of particles released at each coastgrid (daily release)
p_total = np.array(endloc.shape[1])/c_total

# Details needed to seperate islands
channel_gridID = np.array([331, 364, 363, 362, 361, 360])-1
fernandina_gridID = np.arange(332,360)-1

# Location of ports Galapagos Islands (Fig. 1)
lat_ports = [-0.95479, -1.27542, -0.90128, -0.74633, -0.46186]
lon_ports = [-90.96502, -90.48985, -89.61261, -90.30874, -90.27337]

# load example trajectories (Fig. 1)
lons = np.load('input/traj_lon107.npy')
lats = np.load('input/traj_lat107.npy')
lons[lons==0]=np.nan
lats[lats==0]=np.nan



# Preprocessing: reorder coastal grids

Coastal cells need to be ordered by island number, labeled in a clockwise direction starting at the northernmost point and starting with the most eastern islands. 

In [ ]:
# Get all coastal cells in the right order

# extend gridsdataframe to include index and lat/lon of grid center
coastal_cells=np.where(gridnumbermask>0)
gridsdataframe['index_lat'] = coastal_cells[0]
gridsdataframe['index_lon'] = coastal_cells[1]
gridsdataframe['lat'] = lat_release
gridsdataframe['lon'] = lon_release

# Add a new column that will specify the new order of coastal cells
gridsdataframe['index_coastline'] = 0

# start variables
islands = np.array([6,3,5,4,2,9,7,8,1]) #correct order
index_teller = 0

# Loop through all islands

for nisland in islands:
    index_teller += 1
    grid_island = gridsdataframe[gridsdataframe.island_number==nisland]

    # Get the starting point 
    index_start = grid_island.index_lat.argmax() #first most northern grid point
    index_start_table = grid_island.iloc[index_start,:].name # row index of this point in table
    gridsdataframe.loc[index_start_table,'index_coastline']=index_teller # set to first location
    index_teller += 1

    # Get second point (closest in clockwise direction)
    start_lat = grid_island.loc[index_start_table,'index_lat']
    start_lon = grid_island.loc[index_start_table,'index_lon']
    if gridnumbermask[start_lat,start_lon+1]>0: # to the east
        outcome=np.logical_and(gridsdataframe.index_lat==start_lat,gridsdataframe.index_lon==start_lon+1)
        gridsdataframe.loc[outcome,'index_coastline']=index_teller
        new_lat = gridsdataframe.loc[outcome,'index_lat']
        new_lon = gridsdataframe.loc[outcome,'index_lon']
    elif gridnumbermask[start_lat-1,start_lon]>0: # to the south
        outcome=np.logical_and(gridsdataframe.index_lat==start_lat-1,gridsdataframe.index_lon==start_lon)
        gridsdataframe.loc[outcome,'index_coastline']=index_teller
        new_lat = gridsdataframe.loc[outcome,'index_lat']
        new_lon = gridsdataframe.loc[outcome,'index_lon']
    elif gridnumbermask[start_lat,start_lon-1]>0: # to the west
        outcome=np.logical_and(gridsdataframe.index_lat==start_lat,gridsdataframe.index_lon==start_lon-1)
        gridsdataframe.loc[outcome,'index_coastline']=index_teller
        new_lat = gridsdataframe.loc[outcome,'index_lat']
        new_lon = gridsdataframe.loc[outcome,'index_lon']
    else:
        print('something goes wrong with assigning second coastal cell')  

    # Loop through remaining points and assign each closest point that is not already assigned

    for i in range(len(grid_island)-2):
        grid_island = gridsdataframe[gridsdataframe.island_number==nisland]
        index_teller += 1
        distance = ((grid_island[grid_island.index_coastline==0].index_lat - np.array(new_lat))**2 + 
                    (grid_island[grid_island.index_coastline==0].index_lon - np.array(new_lon))**2)
        dist_min = distance.argmin()
        index_next = grid_island[grid_island.index_coastline==0].iloc[dist_min,:].name
        gridsdataframe.loc[index_next,'index_coastline']=index_teller
        new_lat = gridsdataframe.loc[index_next,'index_lat']
        new_lon = gridsdataframe.loc[index_next,'index_lon']
        
# Re-order the transition matrix and the gridsdataframe (re-index) so all islands are in the right order
# add channel and fernandina island

grids = gridsdataframe.copy(deep=True)

index_ordering = np.argsort(np.array(gridsdataframe.index_coastline)-1)
Tmatrix = transitionmatrix.copy()
Tmatrix = Tmatrix[:,index_ordering]
Tmatrix = Tmatrix[index_ordering,:]

# order all other matrixes
TM1 = TM1[:,index_ordering]
TM1 = TM1[index_ordering,:]
TM2 = TM2[:,index_ordering]
TM2 = TM2[index_ordering,:]
TM10 = TM10[:,index_ordering]
TM10 = TM10[index_ordering,:]
TM26 = TM26[:,index_ordering]
TM26 = TM26[index_ordering,:]
TM35 = TM35[:,index_ordering]
TM35 = TM35[index_ordering,:]
TM100 = TM100[:,index_ordering]
TM100 = TM100[index_ordering,:]
TM_all = TM_all[:,index_ordering]
TM_all = TM_all[index_ordering,:]
TM_half = TM_half[:,index_ordering]
TM_half = TM_half[index_ordering,:]

grids = grids.sort_values(by=['index_coastline'],ignore_index=True)
grids.iloc[[channel_gridID],7]=11
grids.iloc[[fernandina_gridID],7]=10

# move Fernandina and channel to the end

new_order = np.arange(0,len(gridsdataframe))
new_order[channel_gridID] = new_order[channel_gridID]+2000
new_order[fernandina_gridID] = new_order[fernandina_gridID]+1000

grids.index_coastline=new_order
index_ordering = np.argsort(np.array(new_order))
Tmatrix = Tmatrix[:,index_ordering]
Tmatrix = Tmatrix[index_ordering,:]

# order all other matrixes
TM1 = TM1[:,index_ordering]
TM1 = TM1[index_ordering,:]
TM2 = TM2[:,index_ordering]
TM2 = TM2[index_ordering,:]
TM10 = TM10[:,index_ordering]
TM10 = TM10[index_ordering,:]
TM26 = TM26[:,index_ordering]
TM26 = TM26[index_ordering,:]
TM35 = TM35[:,index_ordering]
TM35 = TM35[index_ordering,:]
TM100 = TM100[:,index_ordering]
TM100 = TM100[index_ordering,:]
TM_all = TM_all[:,index_ordering]
TM_all = TM_all[index_ordering,:]
TM_half = TM_half[:,index_ordering]
TM_half = TM_half[index_ordering,:]

grids = grids.sort_values(by=['index_coastline'],ignore_index=True)
new_index_coastline = np.arange(1,len(grids)+1)
grids.index_coastline = new_index_coastline


# Fig 1: map

Map of the main islands in the Galapagos Marine Reserve. All port locations are indicated by diamond markers and virtual particle pathways (blue lines) show all existing connections via ocean currents using the MITgcm model simulation from Puerto Ayora (blue diamond) to other coastlines.


In [ ]:
#------ Make map ------#

def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(9, 8),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=False)
    gl.xlabels_bottom = gl.ylabels_left = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

fig, ax = make_map(projection=ccrs.PlateCarree())

extent = [-92, -89, -1.75, 1]
ax.set_extent(extent)

shp = shapereader.Reader('input/GSHHS_f_L1_Galapagos.shp')
for record, geometry in zip(shp.records(), shp.geometries()):
    ax.add_geometries([geometry], ccrs.PlateCarree(), facecolor=(4/255,138/255,112/255),
                      edgecolor='black')

# get colors for each island

norm = colors.Normalize(vmin=0, vmax=max(grids.island_number))
cmap = cm.get_cmap('copper')

legend_colors=[]
for i in range(max(grids.island_number)+1):
    legend_colors.append(colors.to_hex(np.array(cmap(norm(int(i)),bytes=True) )/255))

# Add tourist and port locations

ax.scatter(lon_ports[3],lat_ports[3],s=100,
           facecolor='#6fa8dc',
           edgecolor='k',
           marker='d',
           zorder=202,
           label='Puerto Ayora')
ax.scatter(lon_ports,lat_ports,s=100,
           facecolor='#e69138',
           edgecolor='k',
           marker='d',
           zorder=201,
           label='(Air)ports')

ax.plot([0],[0],color= '#3d85c6',linewidth=0.7, label='Virtual particle pathways from Puerto Ayora')

ax.legend()
    
# Add names of islands
island_names = ['Isabela',
                'Floreana',
                'Española',
                'Santa Cruz',
                'Santa Fé',
                'San Cristóbal',
                'Santiago',
                'Pinta',
                'Marchena',
                'Fernandina',
                'Bolivar Channel']

ax.text(-91.3, -0.85, island_names[0], color='w',fontsize='13', weight='bold')
ax.text(-90.93, -1.4, island_names[1], color='k',fontsize='13', weight='bold')
ax.text(-89.6, -1.47, island_names[2], color='k',fontsize='13', weight='bold')
ax.text(-90.18, -0.53, island_names[3], color='k',fontsize='13', weight='bold')
ax.text(-90.3, -0.93, island_names[4], color='k',fontsize='13', weight='bold')
ax.text(-89.7, -1.1, island_names[5], color='k',fontsize='13', weight='bold')
ax.text(-90.7, -0.17, island_names[6], color='k',fontsize='13', weight='bold')
ax.text(-91.07, 0.55, island_names[7], color='k',fontsize='13', weight='bold')
ax.text(-91.02, 0.30, island_names[8], color='k',fontsize='13', weight='bold')
ax.text(-91.99, -0.25, island_names[9], color='k',fontsize='13', weight='bold')
ax.text(-91.99, -0.62, island_names[10], color='k',fontsize='13', weight='bold')
ax.plot([-91.3, -91.35],[-0.55, -0.36],c='k',linewidth=0.5)

# Add trajectories from Puerto Ayora
for i in range(lons.shape[0]):
    ax.plot(lons[i,:],lats[i,:],color= '#3d85c6',linewidth=0.4, zorder=0, alpha=0.7)
     
plt.savefig('figures/islands_map.png',dpi=300,facecolor='#ffffff')


# Fig 2: Connectivity matrix and beaching sensitivity

The transition matrix (a) giving the probability that a particle starting at a source coastal node (y-axis) arrives at a sink coastal node (x-axis) for $\lambda_B$= 5 days, and the probability (b) that a particle starting at a source coastal node is not beaching within 60 days and therefore "lost" to the ocean, including the sensitivity of this loss to changes in the beaching timescale (colored lines) compared to the reference simulation (black line). The different islands are delimited by horizontal and vertical grey lines.


In [ ]:
# make figure

logmatrix=Tmatrix.copy()
logmatrix[logmatrix==0]=np.nan

island_names = ['Isabela',
                'Floreana',
                'Española',
                'Santa Cruz',
                'Santa Fé',
                'San Cristóbal',
                'Santiago',
                'Pinta',
                'Marchena',
                'Fernandina',
                'Channel']

fig = plt.figure(figsize=(14,9))
gs = fig.add_gridspec(2, 2, width_ratios=[1, 0.5], height_ratios = [0.05,1])

# plot transitionmatrix
ax1 = fig.add_subplot(gs[1,0])
im = ax1.pcolor(logmatrix/1800*100, 
                norm=colors.LogNorm(vmin=0.1, vmax=10),
                cmap='magma',
                shading='auto')

# add colorbar
ax2 = fig.add_subplot(gs[0,0])
cbr = plt.colorbar(im, cax=ax2, extend='max', orientation='horizontal')
cbr.set_label('Probability (%)', fontsize=14)
cbr.ax.tick_params(labelsize=12)
#ax2.set_title('Probability (%)', fontsize=14)

# plot % of particles lost to the ocean
col = plt.cm.copper(np.linspace(0.1,1,6,endpoint=True))

ax3 = fig.add_subplot(gs[1,1])
ngrids=np.arange(0,len(grids))
land_frac5 = 100-np.sum(Tmatrix,axis=1)/p_total*100
land_frac1 = 100-np.sum(TM1,axis=1)/p_total*100
land_frac2 = 100-np.sum(TM2,axis=1)/p_total*100
land_frac10 = 100-np.sum(TM10,axis=1)/p_total*100
land_frac26 = 100-np.sum(TM26,axis=1)/p_total*100
land_frac35 = 100-np.sum(TM35,axis=1)/p_total*100
land_frac100 = 100-np.sum(TM100,axis=1)/p_total*100
ax3.plot(land_frac1,ngrids, c=col[0], linewidth=0.4, label = '$\lambda$ = 1 d')
ax3.plot(land_frac2,ngrids, c=col[1], linewidth=0.4, label = '$\lambda$ = 2 d')
ax3.plot(land_frac5,ngrids, c='k', linewidth=1.3, label = '$\lambda$ = 5 d')
ax3.plot(land_frac10,ngrids, c=col[2], linewidth=0.4, label = '$\lambda$ = 10 d')
ax3.plot(land_frac26,ngrids, c=col[3],linewidth=0.4, label = '$\lambda$ = 26 d')
ax3.plot(land_frac35,ngrids, c=col[4], linewidth=0.4, label = '$\lambda$ = 35 d')
ax3.set_xlabel('Connection to ocean sink (%)', fontsize=14)
ax3.set_ylim([0,len(ngrids)])
ax3.set_xlim([0,100])
ax3.set_yticks([])
ax3.tick_params(axis="x", labelsize=12)

ax4 = fig.add_subplot(gs[0,1])
ax4.axis('off')
leg = ax3.legend(bbox_to_anchor=(1, 1), prop={"size":12}, ncol=2, bbox_transform=ax4.transAxes)

for i,legobj in enumerate(leg.legendHandles):
    if i == 2:
        legobj.set_linewidth(2.0)
    else:
        legobj.set_linewidth(1.0)

# add island separation lines
nempty=np.zeros(len(grids))
check = []
yticks = []
inames = []
for i in range(len(grids)):
    island_number = grids.island_number[i]
    if ~np.any(check==island_number):
        ax1.plot(ngrids,nempty+i,c='#5b5b5b',linewidth=0.8)
        ax1.plot(nempty+i,ngrids,c='#5b5b5b',linewidth=0.8)       
        ax3.plot([0,100],[i,i], c='#000000', linewidth=0.6)
        check.append(island_number)
        yticks.append(i)
        inames.append(island_names[island_number-1])
        
# remove numbers
plt.subplots_adjust(wspace=0.05, hspace=0)
ax1.set_yticks([])
ax1.set_xticks([])

# Add names to axis instead of numbers
yticks.append(ngrids[-1])
for i in range(len(yticks)-1):
    loctext = (yticks[i]+yticks[i+1]-1)/2
    ax1.text(-3, loctext, inames[i], fontsize=12, horizontalalignment='right', verticalalignment='center', rotation = 0)
    ax1.text(loctext, -3, inames[i], fontsize=12, horizontalalignment='center', verticalalignment='top', rotation = -90)
    
ax1.text(-50, ngrids[-1]/2, 'source', rotation = 90, fontsize=14)
ax1.text(ngrids[-1]/2, -50,'sink', fontsize=14) 

ax1.text(3, ngrids[-1]+7, 'a)', fontsize=16)
ax3.text(1, ngrids[-1]+7, 'b)', fontsize=16)

# Save figure
#plt.tight_layout()
#plt.savefig('figures/connectivity_matrix.png',dpi=300,facecolor='#ffffff')


# Fig 3: Source Sink Distribution

Source (a) and sink (c) distribution of macroplastic connectivity between the Galapagos Islands, specifying the relative connectivity to/from another island (green), to/from the same island (yellow), to/from the same location (brown) and to the ocean (blue). The total percentage of particles arriving at each location (node) is shown in panel b.

In [ ]:
# Figure of source and sink distribution

x = np.arange(0,c_total)
fig, ax = plt.subplots(3,1, figsize=(12,12), sharex=True)
island_number = np.array(grids.island_number)

# Sink distribution - stacked bar graph

ocean_frac = 100 - np.sum(Tmatrix,axis=1)/p_total*100
long_frac = []
self_frac = []

for i,island in enumerate(island_number):   
    indx=np.where(island_number != island)
    long_frac.append(np.sum(Tmatrix[i,indx])/p_total*100)
    self_frac.append(Tmatrix[i,i]/p_total*100)
        
short_frac = 100 - ocean_frac - long_frac - self_frac

ax[0].bar(x, long_frac, color='#4e8500', width=1, label = 'to another island')
ax[0].bar(x, self_frac, bottom=long_frac, color='#7b4a18', width=1, label = 'to same location')
ax[0].bar(x, short_frac, bottom=[i+j for i,j in zip(long_frac, self_frac)], color='#f7da8b', width=1, label='to same island')
ax[0].bar(x, ocean_frac, bottom=[i+j+z for i,j,z in zip(short_frac, long_frac, self_frac)], color='#d1f3ff', width=1, label='to ocean')

ax[0].set_xlim([0,c_total-1])
ax[0].set_ylim([0,100])
ax[0].legend(loc='upper left', prop={"size":12}, ncol=2)
for i in yticks[1:-1]:
    ax[0].plot([i,i],[0,100],c='k',linewidth=1)

ax[0].set_title('a) Sink distribution', fontsize=12)
ax[0].set_ylabel('Sink distribution at each node (%)', fontsize=12)
ax[0].set_xticks([])
ax[0].tick_params(axis='x',direction='in') 
    
# Source distribution - stacked bar graph
    
long_frac = []
short_frac = []
self_frac = []
total_arrival = []
for i, island in enumerate(island_number):
    indx_long=np.where(island_number != island)
    indx_short=np.where(island_number == island)
    long_frac.append(np.sum(Tmatrix[indx_long,i]))
    short_frac.append(np.sum(Tmatrix[indx_short,i]))
    self_frac.append(np.sum(Tmatrix[i,i]))
    total_arrival.append(np.sum(Tmatrix[:,i]))

i=6
print('percentage arriving at island ' + island_names[i-1] + ' from elsewhere is ' + str(np.sum(np.array(long_frac)[island_number==i])/np.sum(np.array(total_arrival)[island_number==i])))    
    
long_frac = np.array(long_frac)/np.array(total_arrival)*100
short_frac = np.array(short_frac)/np.array(total_arrival)*100
self_frac = np.array(self_frac)/np.array(total_arrival)*100


ax[2].bar(x, long_frac, color='#4e8500', width=1, label = 'from another island')
ax[2].bar(x, self_frac, bottom=long_frac, color='#7b4a18', width=1, label = 'from same location')
ax[2].bar(x, short_frac, bottom=[i+j for i,j in zip(long_frac, self_frac)], color='#f7da8b', width=1, label='from same island')


ax[2].set_xlim([0,c_total-1])
ax[2].set_ylim([0,100])
ax[2].legend(loc='upper left', prop={"size":12}, ncol=2)
for i in yticks[1:-1]:
    ax[2].plot([i,i],[0,100],c='k',linewidth=1)

ax[2].set_title('c) Source distribution', fontsize=12)
ax[2].set_ylabel('Source distribution at each node (%)', fontsize=12)

# Set ticks

island_names = ['Isabela',
                'Floreana',
                'Española',
                'Santa Cruz',
                'Santa Fé',
                'San Cristóbal',
                'Santiago',
                'Pinta',
                'Marchena',
                'Fernandina',
                'Channel']

ax[2].set_xticks([])
ax[2].tick_params(axis='x',direction='in')

for i in range(len(yticks)-1):
    loctext = (yticks[i]+yticks[i+1]-1)/2
    ax[2].text(loctext, -0.8, inames[i], fontsize=12, horizontalalignment='left', verticalalignment='top', rotation = -30)
   
 
fig.subplots_adjust(hspace=0)    
    
# Figure showing total arrival of particles at each node

ax[1].yaxis.tick_right()
ax[1].set_ylabel('Percentage arriving at each node (%)', fontsize=12)
ax[1].yaxis.set_label_position("right")

ax[1].plot(x,np.sum(Tmatrix,axis=0)/(c_total*p_total)*100,c='k')
ax[1].set_ylim([-0.15, 1.15])
ax[1].plot(x,np.zeros_like(x),c='k',linewidth=0.4)
ax[1].plot(x,np.zeros_like(x)+1,c='k',linewidth=0.4)

ax[1].set_title('b) Total arriving', y=1.0, pad=-17, fontsize=12)

for i in yticks[1:-1]:
    ax[1].plot([i,i],[0,1],c='k',linewidth=1)

plt.tight_layout()
plt.savefig('figures/sourcesink_distribution.png',dpi=300,facecolor='#ffffff')


# Functions to calculate centralities and impact

In [ ]:
# Function to calculate all centralities

class CIs:
    
    def __init__(self, Tmatrix, grids, p_total, lon_ports, lat_ports):
        self.tm = Tmatrix
        self.ptotal = p_total
        self.clon = grids.lon
        self.clat = grids.lat 
        self.lon_civil = lon_ports
        self.lat_civil = lat_ports
          
        # initialize parameters of interest
        self.centralities = {}    
        self.graph = None
        
    def make_graph(self):
        
        # initialize graph
        pgraph = nx.DiGraph()
        pgraphR = nx.DiGraph()
        nodenames = []
        locations = []
        
        # Add nodes
        for i in range(self.tm.shape[0]):
            lon =  self.clon[i]
            lat =  self.clat[i]
            location = (lon,lat)

            pgraph.add_node(i, pos = location)
            pgraphR.add_node(i, pos = locations)

        # Add edges
        for release_node in pgraph.nodes():
            for beach_node in pgraph.nodes():
                weight = self.tm[release_node, beach_node]        
                if weight > 0:
                    weightlog = -np.log(self.tm[release_node, beach_node]/self.ptotal)
                    pgraph.add_edge(release_node,
                                    beach_node,
                                    weightpath = weight,
                                    weightlogpath = weightlog)
                    pgraphR.add_edge(beach_node,
                                    release_node,
                                    weightpath = weight,
                                    weightlogpath = weightlog)
        
        self.graph = pgraph
        self.graphR = pgraphR   
    
    def calculate_betweenness(self):
        
        betweenness = nx.centrality.betweenness_centrality(self.graph, weight = 'weightlogpath')
        self.centralities['betweenness'] = betweenness.values()
        
    def calculate_pagerank(self):

        pagerank = nx.pagerank(self.graph, alpha = 0.85, max_iter = 200, tol = 1e-6, weight = 'weightlogpath')  
        self.centralities['PRin'] = pagerank.values()
        pagerank = nx.pagerank(self.graphR, alpha = 0.85, max_iter = 200, tol = 1e-6, weight = 'weightlogpath')  
        self.centralities['PRout'] = pagerank.values()
    
    def calculate_retention(self):
        
        retention = np.zeros(self.tm.shape[0])

        for i in range(self.tm.shape[0]):
            retention[i] = self.tm[i,i]/self.ptotal
        
        self.centralities['retention'] = retention
            
    def calculate_loss(self):
        
        loss = 1 - np.sum(self.tm,axis=1)/self.ptotal
        self.centralities['loss'] = loss
        
    def calculate_beaching(self):
        
        self.centralities['beaching'] = np.sum(self.tm, axis=1)/self.ptotal - self.centralities['retention']
        
    def calculate_SSI(self):
        
        SSIsink = np.zeros(self.tm.shape[0]) #net sink
        SSIsource = np.zeros(self.tm.shape[0]) #net source
        
        for i in range(self.tm.shape[0]):
            SSIsink[i] = (np.sum(self.tm[:,i]) - np.sum(self.tm[i,:]))/(np.sum(self.tm[:,i]) + np.sum(self.tm[i,:]))
            SSIsource[i] = -1*SSIsink[i]
        self.centralities['SSIsink'] = SSIsink
        self.centralities['SSIsource'] = SSIsource
        
    def calculate_SiD(self): #sink diversity (diversity of outgoing edges)
        
        SiD = np.zeros(self.tm.shape[0])
        total_outward = np.sum(self.tm, axis=1)     
               
        for i in range(self.tm.shape[0]): 
            for j in range(self.tm.shape[0]):
                if self.tm[i,j]>0:
                    SiD[i] += -1*self.tm[i,j]/total_outward[i]*np.log(self.tm[i,j]/total_outward[i])
        
        self.centralities['SiD'] = SiD
        
    def calculate_SoD(self): #source diversity (diversity of incoming edges)

        SoD = np.zeros(self.tm.shape[0])
        total_inward = np.sum(self.tm, axis=0)
        
        for j in range(self.tm.shape[0]):
            for i in range(self.tm.shape[0]):
                if self.tm[i,j]>0:
                    SoD[j] += -1*self.tm[i,j]/total_inward[j]*np.log(self.tm[i,j]/total_inward[j])

        self.centralities['SoD'] = SoD
        
    def get_centralities(self):
        
        self.calculate_retention()
        self.calculate_loss()
        self.calculate_beaching() 
        self.calculate_SSI() 
        self.calculate_SiD()
        self.calculate_SoD()
        
        print('done with centralities')
        
        self.make_graph()
        print('made graph, start betweenness')
        self.calculate_betweenness()
        print('calculate pagerank')
        self.calculate_pagerank()
        
# Function to calculate impact metrics homogeneous initialisation

def calculate_impact(centrality_ranked, Tmatrix, centr_name, iters, threshold, p_total):
    
    Tprob = Tmatrix.copy()
    ocean_source = np.zeros(Tprob.shape[0])
    Tprob=np.vstack((Tprob,ocean_source))
    ocean_sink = p_total - np.sum(Tprob,axis=1)
    Tprob=np.column_stack((Tprob,ocean_sink))
    Tprob = Tprob/p_total
    
    # get order of removal
    removal_order = np.array(centrality_ranked,dtype='int')-1
    removal_order = np.argsort(removal_order)
    
    iterations = np.arange(0,iters)
    benchmark = np.linspace(0,1,Tprob.shape[0])
    
    # diagnostics
    benefit = [0]
    fraction_ocean = [np.nan]
    LBL = [1]
    iterations_metric = [np.nan]
    remove_idx = []

    for b,r in enumerate(removal_order):

        remove_idx.append(r)

        plastic = np.zeros(Tprob.shape[0])+1 # all plastic on land
        plastic[-1]=0 # no plastic in the ocean at start
        total = np.sum(plastic) # total plastic at start

        ocean = []
        removed = []
        land = []

        for i in iterations:
            ocean.append(plastic[-1]/total)
            land.append(np.sum(plastic[:-1])/total)
            removed.append((total-np.sum(plastic))/total)
            plastic[remove_idx] = 0 #remove plastic
            plastic = np.dot(plastic,Tprob)

        benefit.append(removed[-1]-benchmark[b])
        LBL.append(land[-1])
        
        threshold_onland = (1-np.nanmin(land))*threshold # if using left on land to calculate steady state
        f = interp1d((1-np.array(land)), iterations)
              
        #threshold_oceanloss = np.nanmax(ocean)*threshold # if using ocean loss to calculate steady state
        #f = interp1d(ocean, iterations)    
        iterations_metric.append(f(threshold_onland)) #number of iterations needed to reach 'threshold'% of final percentage on land
        
    return benefit, LBL, iterations_metric


# Function to calculate impact metrics homogeneous initialisation - combining best scoring centralities

def calculate_impact_combine(remove_idx, Tmatrix, iters, threshold, p_total):
    
    Tprob = Tmatrix.copy()
    ocean_source = np.zeros(Tprob.shape[0])
    Tprob=np.vstack((Tprob,ocean_source))
    ocean_sink = p_total - np.sum(Tprob,axis=1)
    Tprob=np.column_stack((Tprob,ocean_sink))
    Tprob = Tprob/p_total
    
    iterations = np.arange(0,iters)
    benchmark = np.linspace(0,1,Tprob.shape[0])


    plastic = np.zeros(Tprob.shape[0])+1 # all plastic on land
    plastic[-1]=0 # no plastic in the ocean at start
    total = np.sum(plastic) # total plastic at start

    ocean = []
    removed = []
    land = []
    
    for i in iterations:
        ocean.append(plastic[-1]/total)
        land.append(np.sum(plastic[:-1])/total)
        removed.append((total-np.sum(plastic))/total)
        plastic[remove_idx] = 0 #remove plastic
        plastic = np.dot(plastic,Tprob)

    b = len(remove_idx)
    benefit = removed[-1]-benchmark[b]
    LBL = land[-1]
        
    threshold_onland = (1-np.nanmin(land))*threshold # if using left on land to calculate steady state
    f = interp1d((1-np.array(land)), iterations)
              
    #threshold_oceanloss = np.nanmax(ocean)*threshold # if using ocean loss to calculate steady state
    #f = interp1d(ocean, iterations)    
    iterations_metric = f(threshold_onland) #number of iterations needed to reach 'threshold'% of final percentage on land
        
    return benefit, LBL, iterations_metric

# Function to calculate impact metrics different initial distribution

def calculate_impact_distr(centrality_ranked, Tmatrix, centr_name, iters, threshold, p_total, initial_distribution, coastline_fraction):
    
    Tprob = Tmatrix.copy()
    ocean_source = np.zeros(Tprob.shape[0])
    Tprob=np.vstack((Tprob,ocean_source))
    ocean_sink = p_total - np.sum(Tprob,axis=1)
    Tprob=np.column_stack((Tprob,ocean_sink))
    Tprob = Tprob/p_total
    
    # get order of removal
    removal_order = np.array(centrality_ranked,dtype='int')-1
    removal_order = np.argsort(removal_order)
    remove_idx = removal_order[:coastline_fraction]
    removal_order2 = np.argsort(initial_distribution)
    remove_idx2 = removal_order2[-coastline_fraction:]
    
    # useful
    iterations = np.arange(0,iters)
    coastline = np.linspace(0,1,Tprob.shape[0]) # fraction of coastline cleaned

    # initial distribution - cleaning based on centrality
    plastic = np.copy(initial_distribution)
    plastic[-1] = 0 # make sure there is no plastic in the ocean at start
    total = np.sum(plastic) # total plastic at start
    removed = []

    for i in iterations:
        removed.append((total-np.sum(plastic))/total)
        plastic[remove_idx] = 0 #remove plastic
        plastic = np.dot(plastic,Tprob)
   
    total_removed = removed[-1]
        
    # initial distribution - cleaning if knowing where high MPD
    plastic = np.copy(initial_distribution)
    plastic[-1] = 0 # make sure there is no plastic in the ocean at start
    total = np.sum(plastic) # total plastic at start
    removed = []

    for i in iterations:
        removed.append((total-np.sum(plastic))/total)
        plastic[remove_idx2] = 0 #remove plastic
        plastic = np.dot(plastic,Tprob)
   
    optimal_removed = removed[-1]
    
    
    return total_removed, optimal_removed

# Function to get all centrality rankings

def get_centrality_rankings(transition_matrix, grids, p_total, lon_ports, lat_ports):

    centr_obj = CIs(transition_matrix, grids, p_total, lon_ports, lat_ports)
    centr_obj.get_centralities()
    centr = pd.DataFrame(centr_obj.centralities)
    ranking = centr.rank(ascending=False) #highest centrality should be cleaned first
    
    return centr_obj, centr, ranking
        
        

In [ ]:
# Explanation for finding steady state (comment 1)

n = 1
centr_names = ranking.columns.values.tolist()
centrality_ranked = ranking[centr_names[n]]
print(centr_names[n])
iters = 30
threshold = 0.99
threshold_l = 0.99

Tprob = Tmatrix.copy()
ocean_source = np.zeros(Tprob.shape[0])
Tprob=np.vstack((Tprob,ocean_source))
ocean_sink = p_total - np.sum(Tprob,axis=1)
Tprob=np.column_stack((Tprob,ocean_sink))
Tprob = Tprob/p_total

# get order of removal
removal_order = np.array(centrality_ranked,dtype='int')-1
removal_order = np.argsort(removal_order)

iterations = np.arange(0,iters)
benchmark = np.linspace(0,1,Tprob.shape[0])

# make figures
fig,ax = plt.subplots(2,1, figsize=(8,10))

b = int(len(centrality_ranked)*0.1)
r = removal_order[b]
remove_idx = removal_order[:b]
    
plastic = np.zeros(Tprob.shape[0])+1 # all plastic on land
plastic[-1]=0 # no plastic in the ocean at start
total = np.sum(plastic) # total plastic at start

ocean = []
removed = []
land = []

for i in iterations:
    ocean.append(plastic[-1]/total)
    land.append(np.sum(plastic[:-1])/total)
    removed.append((total-np.sum(plastic))/total)
    plastic[remove_idx] = 0 #remove plastic
    plastic = np.dot(plastic,Tprob)


ax[0].plot(iterations,ocean,c='b',label='% in ocean')
ax[0].plot(iterations,land,c='g',label='% on land')
ax[0].plot(iterations,removed,c='brown', label='% removed')
ax[0].set_ylim([0,1])
ax[0].set_ylabel('fraction')
ax[0].set_xlabel('iteration')
ax[0].set_title('Distribution of particles across reservoirs - 10% repeated cleanup effort')    
    
threshold_oceanloss = np.nanmax(ocean)*threshold
f = interp1d(ocean, iterations)    
ax[0].plot([f(threshold_oceanloss), f(threshold_oceanloss)], [0,1],'--b',label='steady state using ocean curve')

threshold_oceanloss = (1-np.nanmin(land))*threshold_l
f = interp1d((1-np.array(land)), iterations)    
ax[0].plot([f(threshold_oceanloss), f(threshold_oceanloss)], [0,1],'-.g', label='steady state using land curve')

ax[0].legend()

# second panel

b = int(len(centrality_ranked)*0.9)
r = removal_order[b]
remove_idx = removal_order[:b]
    
plastic = np.zeros(Tprob.shape[0])+1 # all plastic on land
plastic[-1]=0 # no plastic in the ocean at start
total = np.sum(plastic) # total plastic at start

ocean = []
removed = []
land = []

for i in iterations:
    ocean.append(plastic[-1]/total)
    land.append(np.sum(plastic[:-1])/total)
    removed.append((total-np.sum(plastic))/total)
    plastic[remove_idx] = 0 #remove plastic
    plastic = np.dot(plastic,Tprob)


ax[1].plot(iterations,ocean,c='b',label='% in ocean')
ax[1].plot(iterations,land,c='g',label='% on land')
ax[1].plot(iterations,removed,c='brown', label='% removed')
ax[1].set_ylim([0,1])
ax[1].set_ylabel('fraction')
ax[1].set_xlabel('iteration')
ax[1].set_title('Distribution of particles across reservoirs - 90% repeated cleanup effort') 

threshold_oceanloss = np.nanmax(ocean)*threshold
f = interp1d(ocean, iterations)    
ax[1].plot([f(threshold_oceanloss), f(threshold_oceanloss)], [0,1],'--b',label='steady state using ocean curve')

threshold_oceanloss = (1-np.nanmin(land))*threshold_l
f = interp1d((1-np.array(land)), iterations)    
ax[1].plot([f(threshold_oceanloss), f(threshold_oceanloss)], [0,1],'-.g', label='steady state using land curve')

ax[1].legend()

plt.savefig('figures/steadystate_calculation_comment1.png',dpi=300,facecolor='#ffffff')

In [ ]:
# Sensitivity to number of particles (comment 1)

from mpl_toolkits.axes_grid1 import make_axes_locatable

logmatrix=Tmatrix.copy()
logmatrix[logmatrix==0]=np.nan

logmatrix_half=TM_half.copy()
logmatrix_half[logmatrix_half==0]=np.nan

fig, ax = plt.subplots(1,2,figsize=(16,8))

# plot transitionmatrix
im1 = ax[0].pcolor(logmatrix/1800*100, 
             norm=colors.LogNorm(vmin=0.1, vmax=10),
             cmap='magma',
             shading='auto')

im2 = ax[1].pcolor(logmatrix_half/900*100, 
             norm=colors.LogNorm(vmin=0.1, vmax=10),
             cmap='magma',
             shading='auto')

divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im2, cax=cax, orientation='vertical')

ax[0].set_title('Transition Matrix with all particles')
ax[1].set_title('Transition Matrix with 0.5 x all particles')

plt.tight_layout()
plt.savefig('figures/nparticles_comment1.png',dpi=300,facecolor='#ffffff')


# Fig 4: Comparison of different centralities - impact

A comparison of the impact metrics described in section 2.5 as a function of the fraction of coastline nodes cleaned for all centrality rankings (colored lines). The benefit metric (a) measures the difference (in %) between the total number of particles removed and the number of particles removed if there would have been zero connectivity between the different nodes. The Left Behind on Land metric (b) indicates how many particles are still on land after steady state is reached. The Iterations metric (c) shows how many iterations where needed to reach steady state and provides an indication for how often one should clean.

In [ ]:
# make figure

centr_obj, centr, ranking = get_centrality_rankings(Tmatrix, grids, p_total, lon_ports, lat_ports)

centr_names = ranking.columns.values.tolist()
coastline = np.linspace(0,1,Tmatrix.shape[0]+1)*100

iters = 30 # should be big enough to always reach steady state
threshold = 0.99 # to find when in steady state

cmap = plt.cm.get_cmap('tab10')
colors = np.linspace(0,1,len(centr_names))
fig, ax = plt.subplots(3,1, figsize=(10,10), sharex='all')

for i,name in enumerate(centr_names):
    
    benefit, LBL, iterations_metric = calculate_impact(ranking[name], Tmatrix, name, iters, threshold, p_total)
    ax[0].plot(coastline, np.array(benefit)*100, color=cmap(colors[i]), label=name)
    ax[1].plot(coastline[1:], np.array(LBL[1:])*100, color=cmap(colors[i]))
    ax[2].plot(coastline[:-1], iterations_metric[:-1], color=cmap(colors[i]), label=name)

fig.subplots_adjust(hspace=0)  
ax[0].legend(bbox_to_anchor=(1.01, 1.0), loc='upper left', prop={"size":12}, ncol=1)
ax[2].set_xlim([0,100])
ax[0].set_ylabel('Benefit (%)')
ax[1].set_ylabel('Left Behind on Land (%)')
ax[2].set_ylabel('Iterations needed to reach steady state')
ax[2].set_xlabel('Fraction of coastline cleaned (%)')

ax[0].text(96, 19.5, 'a)', fontsize=16)
ax[1].text(96, 3.2, 'b)', fontsize=16)
ax[2].text(96, 15.7, 'c)', fontsize=16)
ax[2].set_ylim([0,17])

plt.tight_layout()

plt.savefig('figures/impact_centralities.png',dpi=300,facecolor='#ffffff')

In [ ]:
# make figure to compare combining 4 best scoring centralities (reviewer 3)

#centr_obj, centr, ranking = get_centrality_rankings(Tmatrix, grids, p_total, lon_ports, lat_ports)

centr_names = ranking.columns.values.tolist()
coastline = np.linspace(0,1,Tmatrix.shape[0]+1)*100

iters = 30 # should be big enough to always reach steady state
threshold = 0.99 # to find when in steady state

cmap = plt.cm.get_cmap('tab10')
colors = np.linspace(0,1,len(centr_names))
fig, ax = plt.subplots(3,1, figsize=(10,10), sharex='all')

for i,name in enumerate(centr_names):    
    benefit, LBL, iterations_metric = calculate_impact(ranking[name], Tmatrix, name, iters, threshold, p_total)
    ax[0].plot(coastline, np.array(benefit)*100, color='grey', linewidth=0.7)
    ax[1].plot(coastline[1:], np.array(LBL[1:])*100, color='grey', linewidth=0.7)
    ax[2].plot(coastline[:-1], iterations_metric[:-1], color='grey', linewidth=0.7)


# New order based on 4 centralities
order_RR = np.array(ranking[centr_names[0]],dtype='int')-1
order_RR = np.argsort(order_RR)
order_SSIsink = np.array(ranking[centr_names[3]],dtype='int')-1
order_SSIsink = np.argsort(order_SSIsink)
order_BC = np.array(ranking[centr_names[7]],dtype='int')-1
order_BC = np.argsort(order_BC)
order_PRin = np.array(ranking[centr_names[8]],dtype='int')-1
order_PRin = np.argsort(order_PRin)

nx = np.arange(1,len(remove_idx)+1,dtype='int')

    # make new index and remove duplicates
remove_idx = []
for i in range(len(order_RR)):
    remove_idx.append(order_RR[i])
    remove_idx.append(order_SSIsink[i])
    remove_idx.append(order_BC[i])
    remove_idx.append(order_PRin[i])
remove_idx = list(OrderedDict.fromkeys(remove_idx))

benefit=[]
LBL=[]
iterations_metrix=[]
for i in nx:
    cbenefit, cLBL, citerations_metric = calculate_impact_combine(remove_idx[:i], Tmatrix, iters, threshold, p_total)
    benefit.append(cbenefit*100)
    LBL.append(cLBL*100)
    iterations_metrix.append(citerations_metric)
    
ax[0].plot(coastline[1:], benefit, color='r', label='Combining 4 most promising centralities')
ax[1].plot(coastline[1:], LBL, color='r')
ax[2].plot(coastline[1:], iterations_metrix, color='r')
ax[0].plot(coastline, np.zeros(len(coastline))-10,color='grey',linewidth=0.7,label='Individual centralities')


fig.subplots_adjust(hspace=0)  
ax[0].legend(bbox_to_anchor=(1.01, 1.0), loc='upper left', prop={"size":12}, ncol=1)
ax[2].set_xlim([0,100])
ax[0].set_ylabel('Benefit (%)')
ax[1].set_ylabel('Left Behind on Land (%)')
ax[2].set_ylabel('Iterations needed to reach steady state')
ax[2].set_xlabel('Fraction of coastline cleaned (%)')
ax[0].set_ylim([-1,22])

ax[0].text(96, 19.5, 'a)', fontsize=16)
ax[1].text(96, 3.2, 'b)', fontsize=16)
ax[2].text(96, 15.7, 'c)', fontsize=16)
ax[2].set_ylim([0,17])

plt.tight_layout()
plt.savefig('figures/impact_combined_centralities.png',dpi=300,facecolor='#ffffff')

# Fig 5: Impact limited cleaning effort

The benefit of all centrality node rankings as a function of (a) how much of the initial plastic distribution is left on land and (b)how often the matrix multiplication needs to be performed to reach steady state. The impact metrics are calculated for when 5% (diamond marker) and 10% (star marker) of the coastline would be cleaned.


In [ ]:
# FIGURE - Impact optimalisation full matrix

centr_obj, centr, ranking = get_centrality_rankings(Tmatrix, grids, p_total, lon_ports, lat_ports)

centr_names = ranking.columns.values.tolist()
coastline = np.linspace(0,1,Tmatrix.shape[0]+1)

iters = 30 # should be big enough to always reach steady state
threshold = 0.95 # to find when in steady state

c10=int(0.1*len(coastline))
c5=int(0.05*len(coastline))

cmap = plt.cm.get_cmap('tab10')
colors = np.linspace(0,1,len(centr_names))
fig, ax = plt.subplots(1,2, figsize=(12,6), sharey='all')

for i,name in enumerate(centr_names):
    
    benefit, LBL, iterations_metric = calculate_impact(ranking[name], Tmatrix, name, iters, threshold, p_total)
    ax[0].plot(np.array(LBL[1:c10+1])*100,np.array(benefit[1:c10+1])*100, color=cmap(colors[i]),linewidth=1,zorder=0)
    ax[0].scatter(LBL[c10]*100,benefit[c10]*100, s=150, color=cmap(colors[i]), marker='*',zorder=20)
    ax[0].scatter(LBL[c5]*100,benefit[c5]*100, s=60, color=cmap(colors[i]), marker='D',zorder=20)
    ax[1].plot(iterations_metric[0:c10+1],np.array(benefit[0:c10+1])*100, color=cmap(colors[i]),linewidth=1,label=name,zorder=0)
    ax[1].scatter(iterations_metric[c10],benefit[c10]*100, s=150, color=cmap(colors[i]), marker='*',zorder=20)
    ax[1].scatter(iterations_metric[c5],benefit[c5]*100, s=60, color=cmap(colors[i]), marker='D',zorder=20)

# get some additional legends
ax[0].scatter(-10,0, s=60, c='k', marker='D',label='5% cleanup effort')
ax[0].scatter(-10,0, s=150, c='k', marker='*',label='10% cleanup effort')
    
    
ax[1].legend(fontsize=12)
ax[0].set_xlabel('Left Behind on Land (%)', fontsize=12)
ax[1].set_xlabel('Number of iterations needed to reach steady state', fontsize=12)
ax[0].set_ylabel('Benefit (%)', fontsize=12)
ax[1].set_xlim([0,7])
ax[0].set_xlim([-0.1, 3.5])
ax[0].legend(fontsize=12)
ax[1].set_xticks([1,2,3,4,5,6,7])
ax[0].set_xticks([0,0.5,1,1.5,2,2.5,3])
ax[0].set_ylim([0,14])

fig.subplots_adjust(wspace=0)  

ax[0].text(-0.05, 0.2, 'a)', fontsize=16)
ax[1].text(0.1, 0.2, 'b)', fontsize=16)

plt.savefig('figures/impact_optimisation.png',dpi=300,facecolor='#ffffff')


# Fig 6: Comparison impact with known distribution

The difference between the total removed particle mass if the initial distribution of particles is known and the total removed particle mass when using the SSIsink centrality. The difference is plotted as a function of how clean the coastline is initially (in %). For this calculation, a cleanup effort of 10% is applied and each calculation is repeated 1000 times with randomly distributed particle weight. Outliers are shown with diamond markers.

In [ ]:
# test when cleaning at the 10% highest polluted locations vs. cleaning at 10% highest centrality - sensitivity to % polluted coastline

centr_obj, centr, ranking = get_centrality_rankings(Tmatrix, grids, p_total, lon_ports, lat_ports)

centr_names = ranking.columns.values.tolist()
coastline = np.linspace(0,1,Tmatrix.shape[0]+1)*100

iters = 30 # should be big enough to always reach steady state
threshold = 0.95 # to find when in steady state

coastline_fraction = 0.1 #index of where x% of coastline is cleaned
clean_fraction = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])*100
polluted_fraction = np.array([1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])*100

randomdistr = {}
randomdistr['polluted']=[]
randomdistr['advantage']=[]
randomdistr['effective_removed']=[]
randomdistr['optimal_removed']=[]

name = centr_names[3]

for i,c in enumerate(clean_fraction):
    print(c)
    for k in range(500):
        initial_distribution=np.random.randint(1000, size=(Tmatrix.shape[0]+1))
        where_is_clean = np.random.choice(np.arange(0,Tmatrix.shape[0]+1), int((Tmatrix.shape[0]+1)*c/100), replace=False)
        initial_distribution[where_is_clean]=0
        initial_distribution[-1]=0 #start with no plastic in the ocean

        total_removed, optimal_removed = calculate_impact_distr(ranking[name], Tmatrix, name, iters, threshold, p_total, initial_distribution, int(coastline_fraction*len(coastline)))

        randomdistr['polluted'].append(int(polluted_fraction[i]))
        randomdistr['effective_removed'].append(total_removed)
        randomdistr['optimal_removed'].append(optimal_removed)
        randomdistr['advantage'].append(total_removed - optimal_removed)            
        
impacts = pd.DataFrame(randomdistr)
impacts.advantage = impacts.advantage*100 # in %


In [ ]:
# test using gaussian random field generator - sensitivity to correlation length scale

#centr_obj, centr, ranking = get_centrality_rankings(Tmatrix, grids, p_total, lon_ports, lat_ports)

import pickle
no_data=False #script takes ~20 min to run, outcome is saved as pickle in input file

# generate mask for the coastal locations
coastmask = np.copy(gridnumbermask)
coastmask[coastmask>0]=1
coastmask[coastmask==0]=np.nan

# field size for initial distribution field
x = np.arange(0,len(gridlon[0,:]))
y = np.arange(0,len(gridlat[:,0]))

centr_names = ranking.columns.values.tolist()
coastline = np.linspace(0,1,Tmatrix.shape[0]+1)*100

iters = 30 # should be big enough to always reach steady state
threshold = 0.99 # to find when in steady state

coastline_fraction = 0.1 #cleanup effort
corr_length = np.array([1,2,3,4,5,6,7,8,9,10])*4

randomdistr2 = {}
randomdistr2['correlation_length_scale']=[]
randomdistr2['advantage']=[]
randomdistr2['effective_removed']=[]
randomdistr2['optimal_removed']=[]

name = centr_names[3] # which centrality do we look at

if no_data:
    for i,Lcorr in enumerate(corr_length):
        print(Lcorr)
        model = gs.Gaussian(dim=2, var=1, len_scale=Lcorr/4)

        for k in range(500):              
            # create random initial distribution        
            srf = gs.SRF(model)
            random_field = srf.structured([x, y])
            field = (random_field+np.abs(np.min(random_field)))*coastmask
            initial_distribution = np.reshape(field[field>-1],-1)
            initial_distribution = initial_distribution[grids.Grid_Number-1]
            initial_distribution = np.append(initial_distribution,[0]) #start with no plastic in the ocean

            total_removed, optimal_removed = calculate_impact_distr(ranking[name], Tmatrix, name, iters, threshold, p_total, initial_distribution, int(coastline_fraction*len(coastline)))

            randomdistr2['correlation_length_scale'].append(Lcorr)
            randomdistr2['effective_removed'].append(total_removed)
            randomdistr2['optimal_removed'].append(optimal_removed)
            randomdistr2['advantage'].append(total_removed-optimal_removed)            
        
    impacts2 = pd.DataFrame(randomdistr2)
    impacts2.to_pickle('input/sensitivity_initial_distribution.pkl')
else:
    impacts2 = pd.read_pickle('input/sensitivity_initial_distribution.pkl')

impacts2.advantage = impacts2.advantage*100


In [ ]:
# FIGURE optimal vs. effective removal 

import seaborn as sns

########### random distribution

fig, ax = plt.subplots(1,2, figsize=(12,7))

flierprops = dict(markerfacecolor='0.75', marker='d', markersize=3, linestyle='none')

sns.boxplot(x='polluted', y='advantage',data=impacts, ax=ax[0], color='blue', 
            saturation=0.5, flierprops=flierprops,
            width=0.4)
xvals = np.unique(impacts.polluted)
positions = range(len(xvals))
ax[0].set_ylabel('Advantage (%)',fontsize=12)
ax[0].set_xlabel('Fraction of coastline that is initially polluted (%)',fontsize=12)
for patch in ax[0].artists:
    r, g, b, a = patch.get_facecolor()
    patch.set_facecolor((r, g, b, .3))
ax[0].set_ylim([-100,10])

means = [np.median(impacts[impacts.polluted == xi].advantage) for xi in xvals]
ax[0].plot(positions, means, '--k', lw=2)

############ length scale

flierprops = dict(markerfacecolor='0.75', marker='d', markersize=3, linestyle='none')

sns.boxplot(x='correlation_length_scale', y='advantage',data=impacts2, ax=ax[1], color='blue', 
            saturation=0.5, flierprops=flierprops,
            width=0.4)
xvals = np.unique(impacts2.correlation_length_scale)
positions = range(len(xvals))
ax[1].set_ylabel('Advantage (%)',fontsize=12)
ax[1].set_xlabel('Correlation length scale of initial random distribution (km)',fontsize=12)
for patch in ax[1].artists:
    r, g, b, a = patch.get_facecolor()
    patch.set_facecolor((r, g, b, .3))
ax[1].set_ylim([-100,10])

means = [np.median(impacts2[impacts2.correlation_length_scale == xi].advantage) for xi in xvals]
ax[1].plot(positions, means, '--k', lw=2)

############ SAVE

ax[0].plot([-1,10],[0,0],'k',linewidth=0.3)
ax[1].plot([-1,10],[0,0],'k',linewidth=0.3)
ax[0].set_xlim([-0.5,9.5])
ax[1].set_xlim([-0.5,9.5])

ax[0].text(-0.4, -98, 'a)', fontsize=16)
ax[1].text(-0.4, -98, 'b)', fontsize=16)

plt.savefig('figures/comparison_advantages.png',dpi=300,facecolor='#ffffff')

# Fig 7: Location of optimal removal 

The 10% highest ranked coastline nodes by the centralities discussed in section 2.4. The coastline locations determined by the Retention Rate (a), the SSIsink centrality (d), the betweenness centrality (h) and the PRin centrality (i) are recommended cleanup locations for the Galapagos Islands management that potentially have high impact (section 3.2).


In [ ]:
# FIGURE - Location of 10% effort

c10=int(0.1*len(coastline))
alphas = np.linspace(0.2,1,c10)
cmap = plt.cm.get_cmap('tab10')

minconn = 0
new_matrix = Tmatrix.copy()
new_matrix[new_matrix<minconn]=0
centr_obj, centr, ranking = get_centrality_rankings(new_matrix, grids, p_total, lon_ports, lat_ports)    
centr_names = ranking.columns.values.tolist()

cmap = plt.cm.get_cmap('tab10')
colors = np.linspace(0,1,len(centr_names))

def get_indices(centrality_ranked, centr_name):
    removal_order = np.array(centrality_ranked,dtype='int')-1
    removal_order = np.argsort(removal_order)    
    return removal_order

fig = plt.figure(figsize=(8,11))
gs = fig.add_gridspec(4,3, width_ratios=[1,1,1], wspace=0, hspace=0)
extent = [-92, -89, -1.75, 1]
shp = shapereader.Reader('input/GSHHS_f_L1_Galapagos.shp')

# plot locations
interest_names = [0,1,2,3,4,5,6,7,8,9]
labels = ['a)','b)','c)','d)','e)','f)','g)','h)','i)','j)']
teller=0
for i in range(4):
    for j in range(3):
        if teller <= 9:
            name = centr_names[interest_names[teller]]
            print(name)

            ax = fig.add_subplot(gs[i,j], projection=ccrs.PlateCarree()) 
            ax.set_extent(extent)
            for record, geometry in zip(shp.records(), shp.geometries()):
                ax.add_geometries([geometry], ccrs.PlateCarree(), facecolor=(216/255,216/255,216/255),
                                  edgecolor='black')

            removal_order = get_indices(ranking[name], name)
            ax.scatter(grids.lon[removal_order[:c10]],grids.lat[removal_order[:c10]],
                       s=29,
                       color=cmap(colors[interest_names[teller]]),
                       alpha=alphas,zorder=20)    
            ax.scatter(-1,-1,
                       s=29,
                       color=cmap(colors[interest_names[teller]]),
                       label=labels[teller]+' '+name)
            ax.legend(loc='upper right')
            teller+=1
        
plt.savefig('figures/impact_location.png',dpi=300,facecolor='#ffffff')